# Distributed Model Parallel Training on Amazon SageMaker
In this notebook we will use a Visual transformer to do image classification `horse or human` data from https://laurencemoroney.com/datasets.html. We will download both training and validation dataset provided on the site. 

Note: 
- Kernel: `PyTorch 1.8 Python 3.6 CPU Optimized)`
- Instance Type: `ml.m5.xlarge`

In [117]:
# update sagemaker
!pip install -U sagemaker

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 521 kB 9.0 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.85.0-py2.py3-none-any.whl size=720822 sha256=aa5805fed2aef03bd3b9e4a9d4c194c8a113121a115a92a8498660406ac29e09
  Stored in directory: /tmp/pip-ephem-wheel-cache-o_dehoet/wheels/4d/28/db/508f5f0c9a229fcdaf43d047fc16d21c51fd3e7e6d674d464e
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.84.0
    Uninstalling sagemaker-2.84.0:
      Successfully uninstalled sagemaker-2.84.0
distutils: /opt/conda/include/python3.6m/UNKNOWN
sysconfig: /opt/conda/include/python3.6m
user = False
home = None
root = None
prefix = None


In [1]:
# make sure version is greater or equal to '2.84.0'
import sagemaker
sagemaker.__version__

'2.85.0'

In [2]:
## Download data
!curl -o train.zip https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  142M  100  142M    0     0  96.2M      0  0:00:01  0:00:01 --:--:-- 96.2M


In [3]:
!curl -o validation.zip https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.9M  100 10.9M    0     0  20.3M      0 --:--:-- --:--:-- --:--:-- 20.3M


In [4]:
## Unzip file
import zipfile
with zipfile.ZipFile("train.zip","r") as train_zip_ref:
    train_zip_ref.extractall("data/train")
    
with zipfile.ZipFile("validation.zip","r") as val_zip_ref:
    val_zip_ref.extractall("data/validation")

## Convert images to High Resolution
We will start with converting our images to High Resolution using HuggingFace model [EdsrModel](#https://huggingface.co/eugenesiow/edsr-base) from `super-image` library. 
Please note that this step is optional, and reason for doing this is to mimick the real world image datasets for High Performance Computing, where image size might be in mega bytes.

In [2]:
!pip install datasets super-image
!python3 -m pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 325 kB 8.0 MB/s            
     |████████████████████████████████| 85 kB 109.2 MB/s            
     |████████████████████████████████| 211 kB 110.1 MB/s            
     |████████████████████████████████| 78 kB 109.6 MB/s            
     |████████████████████████████████| 67 kB 104.1 MB/s            
     |████████████████████████████████| 831.4 MB 159.8 MB/s            
     |████████████████████████████████| 84 kB 102.8 MB/s            
     |████████████████████████████████| 67 kB 117.0 MB/s            
     |████████████████████████████████| 43 kB 78.7 MB/s             
     |████████████████████████████████| 40 kB 100.1 MB/s           
INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
     |████████████████████████████████| 312 kB 106.2 MB/s            
     |██████████████

We are using `horse-or-human` dataset from here, which has image size of about `178KB`, we will convert these images to Higher resolution and `resulting size would be close to 2MB`. 

In [3]:
from super_image import EdsrModel, ImageLoader
from PIL import Image
import requests

model = EdsrModel.from_pretrained('eugenesiow/edsr-base', scale=4)

https://huggingface.co/eugenesiow/edsr-base/resolve/main/pytorch_model_4x.pt


In [11]:
import os
from os import listdir
folder_dir = "data/validation/"
for folder in os.listdir(folder_dir):
    folder_path = f'{folder_dir}{folder}'
    for image_file in os.listdir(folder_path):
        path = f'{folder_path}/{image_file}'
        image = Image.open(path)
        inputs = ImageLoader.load_image(image)
        preds = model(inputs)
        ImageLoader.save_image(preds, path)

In [12]:
# quick check on image size for the last image converted by the model.
import os

file_size = os.path.getsize(path)
print("File Size is :", file_size/1000000, "MB")

File Size is : 0.481116 MB


## Optional: Duplicate files to increase number of images for testing purpose in the later sections

In [9]:
# import os
# import shutil

# # repeat the same code for other folders as well.
# source_folder = r"./data/validation/horses/"
# destination_folder = r"./data/validation/horses/"
# i=0
# # fetch all files
# for file_name in os.listdir(source_folder):
#     # construct full file path
#     i=i+1
#     source = source_folder + file_name
#     destination = destination_folder + str(i) + '_' + file_name
#     # copy only files
#     if os.path.isfile(source):
#         shutil.copy(source, destination)
#         print('copied', destination)

copied ./data/validation/horses/1_horse5-235.png
copied ./data/validation/horses/2_horse5-550.png
copied ./data/validation/horses/3_horse1-554.png
copied ./data/validation/horses/4_horse6-153.png
copied ./data/validation/horses/5_horse5-514.png
copied ./data/validation/horses/6_horse3-255.png
copied ./data/validation/horses/7_horse1-455.png
copied ./data/validation/horses/8_horse5-181.png
copied ./data/validation/horses/9_horse2-544.png
copied ./data/validation/horses/10_horse5-002.png
copied ./data/validation/horses/11_horse1-204.png
copied ./data/validation/horses/12_horse1-105.png
copied ./data/validation/horses/13_horse4-102.png
copied ./data/validation/horses/14_horse1-411.png
copied ./data/validation/horses/15_horse6-089.png
copied ./data/validation/horses/16_horse1-510.png
copied ./data/validation/horses/17_horse5-275.png
copied ./data/validation/horses/18_horse4-159.png
copied ./data/validation/horses/19_horse3-484.png
copied ./data/validation/horses/20_horse2-368.png
copied ./

## Upload data to s3

In [3]:
%%time

import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import boto3

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()

prefix = 'horse-or-human'

role = get_execution_role() 
client = boto3.client('sts')
account = client.get_caller_identity()['Account']
print(f'AWS account:{account}')

session = boto3.session.Session()
region = session.region_name
print(f'AWS region:{region}')

AWS account:706553727873
AWS region:us-west-2
CPU times: user 374 ms, sys: 34.8 ms, total: 409 ms
Wall time: 961 ms


In [7]:
from sagemaker.s3 import S3Uploader
# s3_input_data = f's3://{bucket}/{prefix}/data'
s3_train_data = S3Uploader.upload('data/train',f's3://{bucket}/{prefix}/data/train')
s3_val_data = S3Uploader.upload('data/validation',f's3://{bucket}/{prefix}/data/validation')
print('s3 train data path: ', s3_train_data)
print('s3 validation data path: ', s3_val_data)

s3 train data path:  s3://sagemaker-us-west-2-706553727873/horse-or-human/data/train
s3 validation data path:  s3://sagemaker-us-west-2-706553727873/horse-or-human/data/validation


In [4]:
s3_train_data = 's3://sagemaker-us-west-2-706553727873/horse-or-human/data/train'
s3_val_data = 's3://sagemaker-us-west-2-706553727873/horse-or-human/data/validation'

In [10]:
smp_options = {
    "enabled":True,
    "parameters": {                        # Required
#         "microbatches": 4,
        "partitions": 1,                   # Required
        "placement_strategy": "spread",
        "pipeline": "interleaved",
        "optimize": "speed",
        "ddp": True,
    }
}

mpi_options = {
    "enabled" : True,                      # Required
    "processes_per_host" : 8,              # Required
    # "custom_mpi_options" : "--mca btl_vader_single_copy_mechanism none"
}

In [11]:
## Define PyTorch Estimator
metric_definitions=[
                   {'Name': 'train:error', 'Regex': 'train_loss: ([0-9\.]+)'},
                   {'Name': 'validation:error', 'Regex': 'val_loss : ([0-9\.]+)'}
                ]
from sagemaker.pytorch import PyTorch
estimator = PyTorch(entry_point='train_smp.py',
                    source_dir='src',
                    role=role,
                    instance_count=1,
                    instance_type='ml.p3.16xlarge',
                    framework_version="1.10.0",
                    py_version="py38",
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'epochs':5,
                        'batch_size':8, 
                         'lr':3e-5,
                         'gamma': 0.7
                    },
                    distribution={
                        "smdistributed": {"modelparallel": smp_options},
                        "mpi": mpi_options
                    },
                    debugger_hook_config=False,
#                     disable_profiler=True,
                    metric_definitions = metric_definitions,
                    base_job_name="smp-training",
                    max_run=2400,
                   )

In [12]:
%%time
from sagemaker.inputs import TrainingInput

train = TrainingInput(s3_train_data, content_type='image/png',input_mode='File')
val = TrainingInput(s3_val_data, content_type='image/png',input_mode='File')
estimator.fit({'train':train, 'val': val})

2022-04-11 23:13:06 Starting - Starting the training job...ProfilerReport-1649718786: InProgress
...
2022-04-11 23:14:03 Starting - Preparing the instances for training.........
2022-04-11 23:15:34 Downloading - Downloading input data......
2022-04-11 23:16:30 Training - Downloading the training image.......................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-04-11 23:20:15,029 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-04-11 23:20:15,104 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-04-11 23:20:15,111 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-04-11 23:20:15,640 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.8 -m pip install -r requirements.txt
  Attempting uninstall: traitlets
Found existing installation: trai

In [12]:
estimator.model_data

's3://sagemaker-us-west-2-706553727873/pytorch-training-2022-04-04-23-39-49-889/output/model.tar.gz'